### Spark Moive Recommendation
数据集：[MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()
import mlflow
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
type(movies_df)

Out[4]: pyspark.sql.dataframe.DataFrame

In [8]:
movies_df.count()

Out[5]: 58098

In [9]:
#movies_df.show(5)

movies_df.createOrReplaceTempView("movies_df")

display(spark.sql("SELECT * FROM movies_df limit 5"))

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [10]:
#ratings_df.show(5)

ratings_df.createOrReplaceTempView("ratings_df")

display (spark.sql("SELECT * FROM ratings_df limit 5"))

userId,movieId,rating,timestamp
1,307,3.5,1256677221
1,481,3.5,1256677456
1,1091,1.5,1256677471
1,1257,4.5,1256677460
1,1449,4.5,1256677264


In [11]:
links_df.show(5)

links_df.createOrReplaceTempView("links_df")

display (spark.sql("SELECT * FROM links_df limit 5"))

movieId,imdbId,tmdbId
1,0114709,862
2,0113497,8844
3,0113228,15602
4,0114885,31357
5,0113041,11862


In [12]:
#tags_df.show(5)

tags_df.createOrReplaceTempView("tags_df")

display (spark.sql("SELECT * FROM tags_df limit 5"))

userId,movieId,tag,timestamp
14,110,epic,1443148538
14,110,Medieval,1443148532
14,260,sci-fi,1442169410
14,260,space action,1442169421
14,318,imdb top 250,1442615195


In [13]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 1
Minimum number of ratings per movie is 1

In [14]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

6896 out of 22156 movies are rated by only one user

## Part 1: Spark SQL and Data processing

In [16]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [18]:
# %sql
num_users = spark.sql("SELECT count (distinct userID) as num_users FROM ratings")
display(num_users)

num_users
10532


In [19]:
ratings_df.select("userId").distinct().count()

Out[14]: 10532

In [20]:
type(ratings_df.select("userId"))

Out[15]: pyspark.sql.dataframe.DataFrame

### Q2: The number of Movies

In [22]:
#%sql 
num_movies = spark.sql("SELECT count (distinct movieID) as num_movies FROM movies")
display(num_movies)

num_movies
58098


In [23]:
movies_df.select('movieID').distinct().count()

Out[17]: 58098

In [24]:
movies_df.select('movieID').count()

Out[18]: 58098

### Q3:  the number of rates of user

In [26]:
rated_by_users = ratings_df.select('movieID').distinct().count()
print('How many movies are rated by users?', rated_by_users)

How many movies are rated by users? 22156

In [27]:
%sql
SELECT movies.title, movies.genres ,ratings.rating FROM movies left JOIN ratings ON ratings.movieId = movies.movieID WHERE ratings.rating IS null LIMIT 10

title,genres,rating
One-Eyed Monster (2008),Comedy|Horror|Sci-Fi,null
Deep in the Valley (American Hot Babes) (2009),Comedy,null
Love and Lemons (Små citroner gula) (2013),Romance,null
Deathsport (1978),Action|Sci-Fi,null
Narrien illat (1970),Comedy|Drama|Musical,null
Tortured (2008),Crime|Thriller,null
World War II: When Lions Roared (1994),Drama|War,null
Crocodile (2000),Horror|Thriller,null
"Wind Journeys, The (Viajes del viento, Los) (2009)",Drama,null
Apartment 1303 (2007),Horror,null


### Q4: List Movie Genres

In [29]:
%sql
SELECT DISTINCT(genres) FROM movies LIMIT 10

genres
Comedy|Horror
Adventure|Children
Action|Crime|Thriller
Adventure|Animation|Children|Comedy|Fantasy
Comedy|Romance
Action|Adventure|Thriller
Comedy
Action
Comedy|Drama|Romance
Adventure|Children|Fantasy


In [30]:
%sql
SELECT SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 1), '|', -1) as genre FROM movies
UNION
SELECT  SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 2), '|', -1) as genre FROM movies
UNION
SELECT  SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 3), '|', -1) as genre FROM movies
UNION
SELECT  SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 4), '|', -1) as genre FROM movies
UNION
SELECT  SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 5), '|', -1) as genre FROM movies
UNION
SELECT  SUBSTRING_INDEX(SUBSTRING_INDEX(genres, '|', 6), '|', -1) as genre FROM movies
ORDER BY genre;

--This is method I do not like

genre
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


In [31]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, mean, udf, lit, current_timestamp, unix_timestamp, array_contains
extract_genres = udf(lambda x: x.split("|"), ArrayType(StringType()))
movies_df_clean = movies_df.select("movieId", "title", extract_genres("genres").alias("genres"))
#display(movies_df_clean)

movies_df_clean.createOrReplaceTempView("movies_df_clean")

display (spark.sql("SELECT * FROM movies_df_clean limit 5"))

movieId,title,genres
1,Toy Story (1995),"List(Adventure, Animation, Children, Comedy, Fantasy)"
2,Jumanji (1995),"List(Adventure, Children, Fantasy)"
3,Grumpier Old Men (1995),"List(Comedy, Romance)"
4,Waiting to Exhale (1995),"List(Comedy, Drama, Romance)"
5,Father of the Bride Part II (1995),List(Comedy)


In [32]:
genres_result = list(set(movies_df_clean.select('genres').rdd.flatMap(tuple).flatMap(tuple).collect()))
genres_result

Out[21]: ['Sci-Fi',
 'Adventure',
 'Romance',
 'Drama',
 'Animation',
 'Western',
 'Documentary',
 'Thriller',
 'Fantasy',
 'War',
 'Film-Noir',
 'Horror',
 'Action',
 '(no genres listed)',
 'Musical',
 'Children',
 'Crime',
 'Comedy',
 'Mystery',
 'IMAX']

### Q5: Movie for Each Category

In [34]:
genres_result = list(set(movies_df_clean.select('genres').rdd.flatMap(tuple).flatMap(tuple).collect()))
genres_result

Out[22]: ['Sci-Fi',
 'Adventure',
 'Romance',
 'Drama',
 'Animation',
 'Western',
 'Documentary',
 'Thriller',
 'Fantasy',
 'War',
 'Film-Noir',
 'Horror',
 'Action',
 '(no genres listed)',
 'Musical',
 'Children',
 'Crime',
 'Comedy',
 'Mystery',
 'IMAX']

In [35]:
movie_pdf = movies_df.toPandas()
movie_pdf['genres'].str.get_dummies(sep='|').head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
list_of_movie = list(movie_pdf['title'])

## Part2: Spark ALS based approach for training model

In [38]:
#ratings_df.show(10)

ratings_df.createOrReplaceTempView("ratings_df")

display (spark.sql("SELECT * FROM ratings_df limit 5"))

userId,movieId,rating,timestamp
1,307,3.5,1256677221
1,481,3.5,1256677456
1,1091,1.5,1256677471
1,1257,4.5,1256677460
1,1449,4.5,1256677264


In [39]:
movie_ratings=ratings_df.drop('timestamp')

In [40]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [41]:
#movie_ratings.show(10)

movie_ratings.createOrReplaceTempView("movie_ratings")

display (spark.sql("SELECT * FROM movie_ratings limit 10"))

userId,movieId,rating
1,307,3.5
1,481,3.5
1,1091,1.5
1,1257,4.5
1,1449,4.5
1,1590,2.5
1,1591,1.5
1,2134,4.5
1,2478,4.0
1,2840,3.0


### ALS Model and Evaluation

In [43]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [44]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])

In [45]:
# Create ALS model
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [46]:
# 1st print a list of parameters
print(als.explainParams())

alpha: alpha for implicit preference (default: 1.0)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan, current: drop)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False)
intermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK)
itemCol: column name for item ids. Ids must be within the integer value range. (default: item, current: movieId)
maxIter: max number of iterations (>= 0). (default: 10, current: 5)
nonnegative: whether to use nonnegative constraint for least squares (default: False)
numItemBlocks: number of item blocks (default: 10)
numUserBlocks: number of user blocks (default: 10)
predictionCol: prediction column name. (default: prediction)
rank: rank of the factorization (default: 10, current: 10)
ratingCol: column name for ratings (default: rating, current: rating)
regParam: regularization parameter (>= 0). (default: 0.1, current: 0.01)
seed: random seed. (default: -1517157561977538513)
userCol: column name for user ids. Ids must be within the integer value range. (default: user, current: userId)

In [47]:
#Tune model using ParamGridBuilder
# it will take long time in the cv period, so just use few parameter to try 

paramGrid = (ParamGridBuilder()
             .addGrid(als.regParam, [0.01])
             .addGrid(als.rank, [10])
             .addGrid(als.maxIter, [15])
             .build())

# paramGrid = (ParamGridBuilder()
#              .addGrid(als.regParam, [0.01, 0.5, 1, 1.5])
#              .addGrid(als.rank, [10, 15, 20, 25])
#              .addGrid(als.maxIter, [1, 5, 10, 15])
#              .build())

In [48]:
# Define evaluator as RMSE

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [49]:
from pyspark.ml.tuning import CrossValidator
# Build Cross validation 
# Create 5-fold CrossValidator
# it takes too long that I only use 2-fold
cv = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=2)

# Run cross validations
cvModel = cv.fit(training)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [50]:
# Extract the best model selected by CV
best_model = cvModel.bestModel

In [51]:
#Fit ALS model to training data

# specify parameter settings by the best model obtained via CV
print ("**Best Model**")
print ("Rank: ", best_model)
print (" MaxIter: ", str(best_model._java_obj.parent().getMaxIter()))
print (" RegParam:",  best_model._java_obj.parent().regParam())

**Best Model**
Rank: ALSModel: uid=ALS_64e9479a92ef, rank=10
 MaxIter: 15
 RegParam: ALS_64e9479a92ef__regParam

### Model testing

In [53]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [54]:
#Print RMSE 
print ("RMSE = "+str(rmse))

RMSE = 0.7107569994200237

In [55]:
#Extract best model from the tuning exercise using ParamGridBuilder

als_best = ALS(maxIter=15, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als_best.fit(training)

In [56]:
#predictions.show(10)

predictions.createOrReplaceTempView("predictions")

display (spark.sql("SELECT * FROM predictions limit 10"))

userId,movieId,rating,prediction
148,1,4.0,3.645511
148,3,3.0,2.3009744
148,29,3.0,3.8303611
148,36,4.0,4.460527
148,58,4.0,4.0860343
148,64,1.0,1.358746
148,631,1.0,1.8058553
148,653,3.0,2.591094
148,708,2.0,3.4750473
148,710,1.0,1.7080603


### Model apply and see the performance

In [58]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.7107569994200237

In [59]:
alldata.registerTempTable("alldata")

/databricks/spark/python/pyspark/sql/dataframe.py:142: DeprecationWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 "Deprecated in 2.0, use createOrReplaceTempView instead.", DeprecationWarning)

In [60]:
%sql SELECT * FROM alldata LIMIT 10

userId,movieId,rating,prediction
8350,148,4.0,3.3800094
8264,148,1.0,1.1569343
6826,148,3.0,2.296391
6328,148,5.0,0.3956195
5751,148,1.5,1.3959177
165,148,3.0,3.305573
9051,148,1.0,1.6596844
8697,148,2.0,2.9162
7051,148,5.0,2.638159
4539,148,2.0,2.0745254


In [61]:
%sql SELECT * FROM movies JOIN alldata ON movies.movieId=alldata.movieId LIMIT 10

movieId,title,genres,userId,movieId,rating,prediction
148,"Awfully Big Adventure, An (1995)",Drama,8350,148,4.0,3.3800094
148,"Awfully Big Adventure, An (1995)",Drama,8264,148,1.0,1.1569343
148,"Awfully Big Adventure, An (1995)",Drama,6826,148,3.0,2.296391
148,"Awfully Big Adventure, An (1995)",Drama,6328,148,5.0,0.3956195
148,"Awfully Big Adventure, An (1995)",Drama,5751,148,1.5,1.3959177
148,"Awfully Big Adventure, An (1995)",Drama,165,148,3.0,3.305573
148,"Awfully Big Adventure, An (1995)",Drama,9051,148,1.0,1.6596844
148,"Awfully Big Adventure, An (1995)",Drama,8697,148,2.0,2.9162
148,"Awfully Big Adventure, An (1995)",Drama,7051,148,5.0,2.638159
148,"Awfully Big Adventure, An (1995)",Drama,4539,148,2.0,2.0745254


## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [63]:
#recommend 10 movies for each users
user_recs = best_model.recommendForAllUsers(10)
#user_recs.show(10)

user_recs.createOrReplaceTempView("user_recs")

display (spark.sql("SELECT * FROM user_recs limit 10"))

userId,recommendations
1580,"List(List(6528, 8.929522), List(26603, 8.626689), List(3903, 7.6779), List(2880, 7.333238), List(3531, 7.215636), List(6592, 7.1090403), List(4168, 7.106708), List(48856, 7.0374985), List(89302, 7.019793), List(5372, 6.9750633))"
4900,"List(List(7490, 5.669924), List(61071, 5.6437907), List(5517, 5.4704504), List(98361, 5.440694), List(142452, 5.3854213), List(6818, 5.361373), List(57706, 5.336205), List(144656, 5.334746), List(27093, 5.2861395), List(105130, 5.2861395))"
5300,"List(List(7168, 10.145484), List(3531, 9.717326), List(4168, 9.682838), List(7572, 9.576992), List(170777, 9.444838), List(142450, 9.433391), List(7297, 9.387732), List(34552, 9.359207), List(2585, 9.319834), List(26347, 9.309673))"
6620,"List(List(7614, 11.291706), List(3014, 10.776884), List(1571, 10.506861), List(68486, 10.493684), List(77191, 10.115659), List(2237, 9.925403), List(165103, 9.795217), List(7069, 9.756572), List(179135, 9.637023), List(4722, 9.499968))"
7240,"List(List(1856, 12.569382), List(3795, 10.558346), List(6983, 10.270081), List(68959, 10.234553), List(2275, 10.185598), List(443, 10.134381), List(5682, 9.8171015), List(74696, 9.557623), List(7128, 9.541445), List(3920, 9.5098915))"
7340,"List(List(89302, 5.748053), List(2880, 4.6566586), List(6528, 4.6412024), List(74727, 4.593356), List(5347, 4.5202465), List(3531, 4.507254), List(5527, 4.4593387), List(33451, 4.4574757), List(5372, 4.4364786), List(32792, 4.426095))"
7880,"List(List(89302, 14.348543), List(5347, 14.341215), List(77667, 12.817751), List(7043, 11.400675), List(96565, 11.222034), List(5526, 10.481774), List(81132, 10.385958), List(4789, 10.258772), List(5862, 10.1942005), List(109864, 10.178357))"
9900,"List(List(3920, 8.074048), List(3531, 7.7612104), List(1529, 7.5225544), List(85790, 7.520519), List(1925, 7.2822943), List(8809, 7.1825533), List(77, 7.1484237), List(7742, 7.052423), List(2590, 7.004626), List(2880, 6.972172))"
471,"List(List(5515, 6.012226), List(7767, 5.6731067), List(2238, 5.4778233), List(3965, 5.3709736), List(77359, 5.2733574), List(4765, 5.206387), List(8270, 5.091782), List(5875, 5.090572), List(102666, 5.066397), List(7301, 5.0451794))"
1591,"List(List(987, 9.371008), List(8521, 9.238794), List(6528, 8.992801), List(2834, 8.804521), List(8711, 8.586845), List(27800, 8.475794), List(3903, 8.458032), List(148881, 8.457217), List(121372, 8.35376), List(4578, 8.264003))"


In [64]:
user_recs.first()

Out[48]: Row(userId=1580, recommendations=[Row(movieId=6528, rating=8.929521560668945), Row(movieId=26603, rating=8.626688957214355), Row(movieId=3903, rating=7.6778998374938965), Row(movieId=2880, rating=7.333238124847412), Row(movieId=3531, rating=7.215635776519775), Row(movieId=6592, rating=7.109040260314941), Row(movieId=4168, rating=7.10670804977417), Row(movieId=48856, rating=7.037498474121094), Row(movieId=89302, rating=7.0197930335998535), Row(movieId=5372, rating=6.975063323974609)])

In [65]:
user_recs.registerTempTable("als_recs_temp")

In [66]:
# seperate the value of 'recommendations' in user_recs

explode_rec = spark.sql('SELECT userId,\
                                explode(recommendations) AS MovieRec\
                                FROM als_recs_temp')
#explode_rec.show(10)


explode_rec.createOrReplaceTempView("explode_rec")

display (spark.sql("SELECT * FROM explode_rec limit 10"))

userId,MovieRec
1580,"List(6528, 8.929522)"
1580,"List(26603, 8.626689)"
1580,"List(3903, 7.6779)"
1580,"List(2880, 7.333238)"
1580,"List(3531, 7.215636)"
1580,"List(6592, 7.1090403)"
1580,"List(4168, 7.106708)"
1580,"List(48856, 7.0374985)"
1580,"List(89302, 7.019793)"
1580,"List(5372, 6.9750633)"


In [67]:
fianl_recs = spark.sql("SELECT userId,\
                               movieIds_and_ratings.movieId AS movieId,\
                               movieIds_and_ratings.rating AS prediction\
                               FROM als_recs_temp\
                               LATERAL VIEW explode(recommendations) exploded_table AS movieIds_and_ratings")

In [68]:
#fianl_recs.show(10)


fianl_recs.createOrReplaceTempView("fianl_recs")

display (spark.sql("SELECT * FROM fianl_recs limit 10"))

userId,movieId,prediction
1580,6528,8.929522
1580,26603,8.626689
1580,3903,7.6779
1580,2880,7.333238
1580,3531,7.215636
1580,6592,7.1090403
1580,4168,7.106708
1580,48856,7.0374985
1580,89302,7.019793
1580,5372,6.9750633


In [69]:
#Before we recommend the films, we need to filter out those users have not seen yet. Therefore, we need to choose rating = 'null' by join the movie ratings

final_rec = fianl_recs.join(movie_ratings,['userId','movieId'],'left').filter(movie_ratings.rating.isNull())
#display(final_rec)

final_rec.createOrReplaceTempView("final_rec")

display (spark.sql("SELECT * FROM final_rec LIMIT 5"))

userId,movieId,prediction,rating
1580,6528,8.929522,null
1580,26603,8.626689,null
1580,3903,7.6779,null
1580,2880,7.333238,null
1580,3531,7.215636,null


In [70]:
final_rec.registerTempTable("final_rec")
movies_df.registerTempTable("movies_df")

### Find recommend films for userid = 575

In [72]:
%sql
SELECT userId,
       title
FROM final_rec t1
LEFT JOIN movies_df t2
ON t1.movieId = t2.movieId
WHERE t1.userId=575
LIMIT 10

userId,title
575,Oklahoma! (1955)
575,Wild Bill (1995)
575,Gentlemen of Fortune (Dzhentlmeny udachi) (1972)
575,Brother (Hermano) (2010)
575,Trouble in Paradise (1932)
575,Possession (1981)
575,Kill List (2011)
575,"Beast of War, The (Beast, The) (1988)"
575,Jump Tomorrow (2001)
575,Before the Rain (Pred dozhdot) (1994)


### Find recommend films for userid = 273

In [74]:
%sql
SELECT userId,
       title
FROM final_rec t1
LEFT JOIN movies_df t2
ON t1.movieId = t2.movieId
WHERE t1.userId=273
LIMIT 5

userId,title
273,Start the Revolution Without Me (1970)
273,Mina Tannenbaum (1994)
273,"Boost, The (1988)"
273,"Ordet (Word, The) (1955)"
273,All the Vermeers in New York (1990)


## Find the similar moives for moive with id: 463, 471

In [76]:
# 1st extract productFeatures matrix
# The productFeatures matrix will be used to create an item-item collaborative filtering recommendation model
from pyspark.mllib.recommendation import ALS
import math

model_a = ALS.train(movie_ratings, rank=10, iterations=15,lambda_=0.01)
model_a.productFeatures().count()

Out[55]: 22156

In [77]:
# look at the feature vector of movie 463
movie_feature = model_a.productFeatures().lookup(471)[0]

In [78]:
# Next define cosine similarity function to measure movie similarity
def cosineSimilarity(vec1, vec2):
  return vec1.dot(vec2) / (LA.norm(vec1) * LA.norm(vec2))

In [79]:
# Assigns the movies title file
movies_file = os.path.join("/FileStore/tables/", 'movies.csv')
movies_sc = sc.textFile(movies_file)

movies_sc_header = movies_sc.take(1)[0]

movies_data = movies_sc.filter(lambda line: line!=movies_sc_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()

movies_titles = movies_data.map(lambda x: (int(x[0]),x[1]))

In [80]:
movies_sc_header

Out[59]: 'movieId,title,genres'

In [81]:
movies_data

Out[60]: PythonRDD[3466] at RDD at PythonRDD.scala:58

In [82]:
movies_titles

Out[61]: PythonRDD[3467] at RDD at PythonRDD.scala:58

In [83]:
# Build similarity matrix for movieid 471 using the product features matrix

similarMovies = model_a.productFeatures().map(lambda products:(products[0],
                                        cosineSimilarity(np.asarray(products[1]), movie_feature))).join(movies_titles).map(lambda r: (r[1][1], r[1][0], r[0]))

# Sort the top 10 most similar movies descendingly by cosine similarity measure
# similarMovies.takeOrdered(11, key=lambda x: -x[1])

In [84]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors, VectorUDT

In [85]:
a = best_model.itemFactors
# display(a.cache())

a.createOrReplaceTempView("a")

display (spark.sql("SELECT * FROM a limit 5"))

id,features
10,"List(-0.7070604, -0.3937841, -0.7354741, -0.5316492, -0.23054606, -0.9362612, 0.4878864, 0.181011, 0.12492501, -0.32780373)"
20,"List(-0.2865907, -0.38920316, -0.757129, -0.41535524, -0.33387962, -0.78734565, 0.74777204, -0.42470917, -0.5602824, -0.50083333)"
30,"List(-0.41119733, -0.6554364, -1.120149, -0.05089784, 0.2785565, -1.4204834, -0.45307264, -0.91303414, 0.8519034, -0.24168478)"
40,"List(-0.50368834, -0.6089162, -0.34873757, -0.42284888, 0.29128578, -1.0997701, 0.53955245, -0.5642491, 1.1535795, 0.067440234)"
50,"List(-0.60370475, -1.0662308, -0.853213, -0.5079328, -0.010333406, -1.0317204, 0.23053925, 0.22187883, 1.0357325, -0.16144118)"


In [86]:
a.registerTempTable("movie_on_movie")

In [87]:
%sql
SELECT features FROM movie_on_movie WHERE id = 471

features
"List(-0.73946416, -1.03179, -0.83905196, -0.6525196, -0.3816911, -0.88358724, -0.47698575, -0.15836999, 0.36126232, -0.6475737)"


In [88]:
%sql
SELECT * FROM ratings WHERE movieId = 463 LIMIT 10

userId,movieId,rating,timestamp
114,463,4,973377486
758,463,3,874853655
1927,463,2,900074111
2481,463,3,899667729
2629,463,2.5,1070005270
3246,463,3,862157039
3832,463,3,953511244
4796,463,2,887467693
5276,463,4,945487923
5323,463,3,961007786


In [89]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",seed=12345, bucketLength=1.0)
#a.printSchema()
#change features columns into dense vector
to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
data = a.select("id", to_vector("features").alias("features"))
#data.printSchema()

In [90]:
model = brp.fit(data)
model.transform(data)

Out[67]: DataFrame[id: int, features: vector, hashes: array<vector>]

In [91]:
model.approxNearestNeighbors(data, Vectors.dense([-0.73946416, -1.03179, -0.83905196, -0.6525196, -0.3816911, -0.88358724, -0.47698575, -0.15836999, 0.36126232, -0.6475737]), 6).collect()

Out[93]: [Row(id=471, features=DenseVector([-0.7395, -1.0318, -0.8391, -0.6525, -0.3817, -0.8836, -0.477, -0.1584, 0.3613, -0.6476]), hashes=[DenseVector([-1.0])], distCol=2.6779069095950976e-08),
 Row(id=6296, features=DenseVector([-0.5401, -1.0054, -0.821, -0.4597, -0.3949, -1.0113, -0.3629, -0.1277, 0.4759, -0.4724]), hashes=[DenseVector([-1.0])], distCol=0.3901452368861754),
 Row(id=97057, features=DenseVector([-0.5904, -0.9122, -0.7997, -0.6033, -0.2024, -1.017, -0.2256, -0.3149, 0.2738, -0.5288]), hashes=[DenseVector([-1.0])], distCol=0.4470437260221147),
 Row(id=3476, features=DenseVector([-0.986, -1.008, -0.8233, -0.5235, -0.3749, -0.8585, -0.5189, -0.3369, 0.684, -0.3538]), hashes=[DenseVector([-1.0])], distCol=0.5504438482643557),
 Row(id=1059, features=DenseVector([-0.7223, -0.6745, -0.8458, -0.5371, -0.2055, -0.8023, -0.2337, -0.3719, 0.5332, -0.821]), hashes=[DenseVector([-1.0])], distCol=0.5859705231399436),
 Row(id=4346, features=DenseVector([-0.649, -0.8289, -0.8635, -0.647, -0.1975, -0.9109, -0.2489, -0.5237, 0.6113, -0.4233]), hashes=[DenseVector([-1.0])], distCol=0.6188139425663551)]

In [92]:
# similar moives for moive with id: 471

In [93]:
%sql
SELECT * FROM movies
WHERE movieId IN (6296,97057,3476,1059,4346)

movieId,title,genres
1059,William Shakespeare's Romeo + Juliet (1996),Drama|Romance
3476,Jacob's Ladder (1990),Horror|Mystery
4346,Bride of the Wind (2001),Drama|Musical|Romance
6296,"Mighty Wind, A (2003)",Comedy|Musical
97057,Kon-Tiki (2012),Adventure|Documentary|Drama


In [94]:
%sql
SELECT features FROM movie_on_movie WHERE id = 463

features
"List(0.64177823, -0.22756293, -1.3410357, 0.8559743, -0.5571711, -1.2529395, 1.1197073, -1.0625343, 0.92473656, 0.6378647)"


In [95]:
model.approxNearestNeighbors(data, Vectors.dense([0.93929714, 0.015614069, -0.3408886, 0.3818301, 0.19762212, -1.4255825, 0.99496984, -0.065754086, 0.43202916, -0.8621043]), 6).collect()

Out[70]: [Row(id=3987, features=DenseVector([0.4357, -0.2675, -0.0643, -0.1631, -0.0658, -1.0791, 0.8362, 0.0065, 0.3355, -1.0788]), hashes=[DenseVector([-1.0])], distCol=0.9916600277390856),
 Row(id=5321, features=DenseVector([0.3429, -0.1578, -0.8135, -0.3116, 0.1501, -1.3535, 0.5879, -0.1207, 0.0999, -0.7233]), hashes=[DenseVector([-1.0])], distCol=1.1814422906931692),
 Row(id=49007, features=DenseVector([0.2805, 0.1314, -0.1589, -0.3341, -0.2482, -1.1048, 0.7964, -0.2131, 0.8589, -0.7289]), hashes=[DenseVector([-1.0])], distCol=1.24729211678601),
 Row(id=554, features=DenseVector([0.1671, -0.3045, -0.322, 0.4282, 0.0049, -0.7048, 1.02, 0.3537, -0.0589, -0.7803]), hashes=[DenseVector([-1.0])], distCol=1.2969541944425715),
 Row(id=7276, features=DenseVector([0.3005, -0.0256, -0.212, -0.4027, 0.2217, -0.8943, 0.7784, -0.0263, 0.4209, -0.2767]), hashes=[DenseVector([-1.0])], distCol=1.3099385736507838),
 Row(id=7224, features=DenseVector([0.32, -0.1144, -0.0263, 0.3018, -0.2104, -0.6419, 0.3948, -0.0621, 0.0535, -1.018]), hashes=[DenseVector([-1.0])], distCol=1.3469792969096368)]

In [96]:
# similar moives for moive with id: 463

In [97]:
%sql
SELECT * FROM movies
WHERE movieId IN (5321,49007,554,7276,7224)

movieId,title,genres
554,Trial by Jury (1994),Crime|Drama|Thriller
5321,"Triumph of Love, The (2001)",Comedy
7224,"Boy with Green Hair, The (1948)",Children|Drama
7276,Hell's Kitchen (1998),Drama
49007,Arabesque (1966),Adventure|Drama|Romance|Thriller


Based on the above, we obtain the 5 movies that are most similar to movie with id: 471. They are:
William Shakespeare's Romeo + Juliet (1996),
"Jacob's Ladder (1990)',
'Bride of the Wind (2001)',
'Stop Making Sense (1984)',
'Mighty Wind, A (2003)',
'Kon-Tiki (2012)'.